We install and import the libraries needed

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 

I import the data from excel into Pandas

In [3]:
data=pd.read_excel("../data/Data_BC_GlobalDelivery (1).xlsx")

In [171]:
data.head()

,journey_id,user_id,driver_id,icon,start_type,start_at,start_lat,start_lon,end_at,end_lat,...,price_distance,price_duration,distance,duration,cost,cost_distance,cost_duration,source,driver_score,rider_score
0,23a1406fc6a11d866e3c82f22eed4d4c,0e9af5bbf1edfe591b54ecdfd7e91e26,583949a89a9ee17d19e3ca4f137b6b4c,executive,asap,2010-11-16 16:44:52.000002,-12.139835,-77.023560,2010-11-16 17:29:56.999996,-12.055370,...,3626.0,195.0,11331.0,234.0,0.0,0.0,0.0,iPhone,5.0,5.0
1,85b7eabcf5d84e42dc7629b7d27781af,56772d544fdfa589a020a1ff894a86f7,d665fb9f75ef5d9cd0fd89479380ba78,executive,reserved,2010-09-11 23:55:00.000002,-12.089954,-76.926270,2010-09-12 01:07:01.999998,-12.022232,...,7665.0,562.0,30270.0,715.0,6173.0,5756.0,417.0,iPhone,4.0,5.0
2,2254715f0f569e3173ec61f16cbb8341,56772d544fdfa589a020a1ff894a86f7,baacf396f773709519bbde35a5eab861,executive,reserved,2010-08-10 13:00:00.000003,-12.022419,-77.106506,2010-08-10 14:26:51.000003,-12.089802,...,7393.0,1001.0,29080.0,1201.0,5875.0,5175.0,700.0,iPhone,NaN,4.0
3,12432d83b49393bf25164673212b8295,56772d544fdfa589a020a1ff894a86f7,e1332f68e81526e498e4d845233a6d7d,executive,reserved,2010-08-08 12:09:59.999996,-12.089958,-76.926208,2010-08-08 12:48:02.000002,-12.021523,...,7112.0,0.0,27230.0,0.0,5340.0,5340.0,0.0,iPhone,5.0,5.0
4,2e51db3a8a77aa594aa686ff19fe6036,56772d544fdfa589a020a1ff894a86f7,d665fb9f75ef5d9cd0fd89479380ba78,executive,asap,2010-05-16 03:58:58.999996,-12.113472,-77.043854,2010-05-16 04:40:34.999997,-12.091087,...,5519.0,193.0,17247.0,232.0,4282.0,4139.0,143.0,iPhone,5.0,5.0


### Exploring and analyzing drivers data

In [4]:
drivers=data.driver_id.unique()

In [5]:
len(drivers)

169

I drop the records for which we dont have a driver_id as they wont be useful for the analysis

In [6]:
data=data[data.driver_id.notna()]

In [7]:
data.reset_index(inplace=True)

In [26]:
data=data.drop("index",axis=1)

In [9]:
data2=data[["driver_id","journey_id","price","cost","end_state","driver_score","rider_score"]]

In [10]:

driv_summary=data2.groupby(["driver_id","end_state"]).agg({"journey_id":"count","price":"sum","cost":"sum","driver_score":"mean","rider_score":"mean"})

In [11]:
driv_summary.reset_index(inplace=True)

In [12]:
driver_canceling=driv_summary[driv_summary.end_state=="driver cancel"]

There are 168 different riders, of which 65 have canceled their journeys ("driver cancel") at least once

At this point I have to decide from which amount of cancelations made by the drivers we can consider them as low quality drivers as if they only have a few number of cancelations it may be due to extraordinary causes. 

For this i can only focus on absolute numbers, but instead I will have to look at the % of journeys they have cancelled. (its not the same to cancel 8 out of 300 that 2 out of 15. Therefore we first need to check the % cancelation.

In [13]:
driver_canceling.set_index("driver_id",inplace=True)

In [14]:
driver_canceling["total_journeys"]=0

/var/folders/tr/7ry1klqj22sglvfj6nq3g5kw0000gn/T/ipykernel_57870/2160433506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_canceling["total_journeys"]=0


In [17]:
journeys=data.groupby("driver_id").count()

In [19]:
driver_canceling=driver_canceling.assign(total_journeys=journeys["journey_id"])

In [21]:
driver_canceling["percentage cancelled"]=(driver_canceling.journey_id/driver_canceling.total_journeys)*100

I am going to classify as low quality drivers those with a percentage of trips cancelled by them greater than 8%

In [31]:
low_quality_drivers=driver_canceling.index[driver_canceling["percentage cancelled"]>=8]

In [74]:
low_quality_drivers=list(low_quality_drivers)

In [79]:
medium_quality_drivers=driver_canceling.index[(driver_canceling["percentage cancelled"]>=4)&(driver_canceling["percentage cancelled"]<=8)]

In [80]:
medium_quality_drivers=list(medium_quality_drivers)

In [81]:
high_quality_drivers=driver_canceling.index[driver_canceling["percentage cancelled"]<4]

In [82]:
high_quality_drivers=list(high_quality_drivers)

#### Now Im going to focus on the rating of the drivers

In [28]:
ratings=data[["driver_id","rider_score","driver_score"]]

In [30]:
ratings=ratings.groupby("driver_id").mean()

We are going to set as:

low quality drivers those that have a rating below 3.5, 


medium quality those with rating above 3.5 and below 4.5 


high quality those with rating higher than or equal to 4.5

In [174]:
for i in range(len(ratings.index)):      

    if ratings.driver_score[i]<3.5:
        
        low_quality_drivers.append(ratings.index[i])       

In [179]:
for i in range(len(ratings.index)):      

    if ratings.driver_score[i]<4.5 and ratings.driver_score[i]>=3.5 and ratings.driver_score[i] not in low_quality_drivers:
        
        medium_quality_drivers.append(ratings.index[i])       

In [87]:
for i in range(len(ratings.index)):      

    if ratings.driver_score[i]>=4.5 and ratings.driver_score[i] not in medium_quality_drivers and ratings.driver_score[i] not in low_quality_drivers:
        
        high_quality_drivers.append(ratings.index[i])  

#### We check "failure" and "not found" trips

For these type of trips we see that not found ones are due to the app not finding a driver and failure might be due to a fail in the app. Therefore thse wont be relevant right now to classify riders

#### Number of trips

Now we are going to classify drivers according to the time they work for us and the amount of profit (price-cost) they report on average to the company and we will classify them in low_productibity_drivers, medium_productibity_drivers and high_productibity_drivers

-low_productibity_drivers below the 30th percentile

-medium_productibity_drivers below the 60th percentile and above the 30th 

-high_productibity_drivers above the 75th percentile

In [109]:
journeys.journey_id.quantile(0.30)

9.0

In [118]:
low_productibity_drivers=journeys.index[journeys.journey_id<journeys.journey_id.quantile(0.30)]

In [121]:
low_productibity_drivers=list(low_productibity_drivers)

In [102]:
journeys.journey_id.median()

25.0

In [172]:
journeys.journey_id.mean()

117.41666666666667

In [132]:
medium_productibity_drivers=journeys.index[(journeys.journey_id<journeys.journey_id.quantile(0.50)) & (journeys.journey_id>=journeys.journey_id.quantile(0.30))]

In [168]:
medium_productibity_drivers=list(medium_productibity_drivers)

In [104]:
journeys.journey_id.quantile(0.75)

89.5

In [137]:
high_productibity_drivers=journeys.index[journeys.journey_id<journeys.journey_id.quantile(0.75)]

In [138]:
high_productibity_drivers=list(high_productibity_drivers)

Now Im going to calculate the average duration per trip and the average profit they make per trip as well as the total amount

In [140]:
trips=data[["driver_id","price","cost","duration","journey_id"]]

In [141]:
trips=trips.groupby("driver_id").agg({"price":"sum","cost":"sum","duration":"sum","journey_id":"count"})

In [143]:
trips["profit"]=trips.price-trips.cost

In [145]:
trips["avg_profit"]=trips.profit/trips.journey_id

In [153]:
loose_money=trips[trips.profit<0]

We see there are drivers that eventhough might have a considerable amount of trips, are loosing money, therefore we will classify them as low_productivity_drivers

I will classify as medium those with an average profit below the 0.5 quantile and above 0 and as high those between the 50th and 100th quartiles

In [155]:
for i in loose_money.index:
    
    if i not in low_productibity_drivers:
        
        low_productibity_drivers.append(i)

In [156]:
trips["avg_profit"].quantile(0.50)

676.7516556291391

In [169]:
for i in range(len(trips.index)):
    
    if trips["avg_profit"][i]<676.7516556291391  and trips.profit[i]>0:
                                                        
        
        medium_productibity_drivers.append(i)      

In [170]:
for i in range(len(trips.index)):
    
    if trips["avg_profit"][i]>=trips["avg_profit"].quantile(0.50):
                                                        
        
        high_productibity_drivers.append(i)
        

## Exploring and analyzing clients data

#### Rating scores

In the same way as we used drivers scores to calsify them, im going to use the "rider_score" to classify clients. In this case we will use the same metrics:

low_quality_riders <3.5

3.5<=medium_quality_riders<4.5

4.5<=high_quality_riders


In [176]:
low_quality_riders=[]
medium_quality_riders=[]
high_quality_riders=[]

In [177]:
for i in range(len(ratings.index)):      

    if ratings.rider_score[i]<3.5:
        
        low_quality_riders.append(ratings.index)   

In [184]:
for i in range(len(ratings.index)):      

    if ratings.rider_score[i]<4.5 and ratings.rider_score[i]>=3.5 :#and ratings.rider_score[i] not in low_quality_riders:
        
        medium_quality_riders.append(ratings.index[i])    

In [182]:
for i in range(len(ratings.index)):      

    if ratings.driver_score[i]>=4.5 and ratings.driver_score[i] not in medium_quality_drivers and ratings.driver_score[i] not in low_quality_drivers:
        
        high_quality_drivers.append(ratings.index[i])  

#### Average and total profit and riders made by users

Now Im going to check the amount of rides each customer has done as well as the profits we made with each customer and the average profit their trips report to the company 

In [197]:
riders=data[["user_id","journey_id","end_state","duration","price","cost","distance"]]

In [198]:
riders=riders.groupby("user_id").agg({"journey_id":"count","end_state":"count","duration":"sum","cost":"sum","price":"sum","distance":"sum"})

In [194]:
riders.journey_id.max()

630

In [200]:
riders["profit"]=riders.price-riders.cost

In [206]:
riders["avg_profit"]=riders.profit/riders.journey_id

In [207]:
riders

,journey_id,end_state,duration,cost,price,distance,profit,avg_profit
user_id,,,,,,,,
00cb8ad6a0f7214d002e3ded6b7c9b80,7,7,1723.0,22246.0,31587.0,88349.0,9341.0,1334.428571
00cb8ad6a0f7214d002e3ded6b7de012,7,7,5722.0,13851.0,17782.0,42793.0,3931.0,561.571429
00cb8ad6a0f7214d002e3ded6b7eb91b,1,1,406.0,4238.0,5368.0,15716.0,1130.0,1130.000000
0194b4a5c9e41bfd35f9168423f7b7b4,2,2,0.0,0.0,0.0,0.0,0.0,0.000000
0194b4a5c9e41bfd35f9168423fa2857,30,30,12231.0,31359.0,50935.0,125130.0,19576.0,652.533333
...,...,...,...,...,...,...,...,...
ff1cb10b9edf79369742a37ce5d8d6f6,1,1,76.0,1700.0,1700.0,4776.0,0.0,0.000000
ff5c924e0b630fd7c019a4234053c385,1,1,61.0,2887.0,4441.0,13720.0,1554.0,1554.000000
ff5c924e0b630fd7c019a42340593ee2,1,1,193.0,1700.0,1700.0,3370.0,0.0,0.000000


In [202]:
riders.profit.quantile(0.25)

938.0

In [203]:
riders.profit.quantile(0.5)

3174.0

In [204]:
riders.profit.quantile(0.75)

11078.0

In [208]:
riders.avg_profit.quantile(0.25)

519.3333333333334

In [209]:
riders.avg_profit.quantile(0.5)

1034.0

In [210]:
riders.avg_profit.quantile(0.75)

1650.0

from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)
     
     
# driver code
